In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
%cd ../drive/My Drive/hw2ml

In [ ]:
#insert the kaggle json file in the content directory before running this cell
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d ibrahimserouis99/one-piece-image-classifier

In [ ]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('one-piece-image-classifier.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()